# Text Database Notebook
I made this notebook to explore the MySQL Python Connector. Markdown cells have
been added for the team's future reference.
### WARNING: Do not 'RUN ALL' cells! 
### This notebook contains cells that remove databases, papers, writes data, etc.
### For exploration purposes only! 

In [1]:
import mysql.connector

# Enter MySQL Password
Password will be input this way to avoid being exposed in the code.

In [2]:
password = input("Enter your database password: ")

# Initial Connection
This cell connects us to MySQL. Change the host and username as needed.

In [3]:
text_db = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password=password,
)

# Initializing the MySQL cursor
This cursor allows us to perform MySQL operations using Python.

In [4]:
mycursor = text_db.cursor()

# Create a database
This cell creates a new database. Change the database name as needed.

In [9]:
mycursor.execute("CREATE DATABASE technical_database")

# Drop a database
### (Be careful with this cell!)
This cell removes an existing database. Change name as needed (the cell below outputs a list of existing databases)

In [7]:
mycursor.execute("DROP DATABASE technical_db")

# List all existing databases
This cell displays all MySQL databases.

In [5]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)

('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('technical_db',)
('world',)


# Connecting to Database
This connects us to a specific database. Change the database name as needed (you 
can choose from the list of databases output by the cell above)

In [6]:
text_db = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password=password,
    database="technical_db"
)

In [7]:
mycursor = text_db.cursor()

# Drop a table from the database
### (Be careful with this cell!)

This removes a table from the database that we are connected to. Change the table name as needed.

In [8]:
mycursor.execute("DROP TABLE papers")

# Create a sample table for papers
This creates a database table called 'papers' with columns for primary key (auto incremented), title, author, and chunk (consisting of 255 chars at most)

In [9]:
mycursor.execute("""
    CREATE TABLE papers (
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(255),
        author VARCHAR(255),
        chunk TEXT(255)
    )
""")

# Show all tables in database
This cell lists all the tables in the database that we're connected to.

In [8]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('chunks',)
('papers',)


# See current directory
We're about to work with files (Tesseract output .txt files), so we need to check our current working directory

In [9]:
import os
print(os.getcwd())

c:\Users\USER\Documents\OJTChatbotBEIC\MySQL


# Upload an extracted paper into the database
The paper is divided into chunks (up to length 255) and stored into the database, 
along with metadata such as title and author name

In [12]:
# Read the entire file content
with open('../Tesseract/Extracted.txt', 'r', encoding="utf-8", errors='ignore') as file:
    file_content = file.read()

# Split the content into smaller strings (up to 255 characters)
max_length = 255
split_content = [file_content[i:i + max_length] for i in range(0, len(file_content), max_length)]

title = "A CASE STUDY OF UNDERSTANDING THE BONAPARTE BASIN USING UNSTRUCTURED DATA ANALYSIS WITH MACHINE LEARNING TECHNIQUES"
authors = "A.N.N. Sazali, N.M. Hernandez, F. Baillard, K.G. Maver"

# Insert each smaller string into the database
query = "INSERT INTO papers (title, author, chunk) VALUES (%s, %s, %s)"
for content in split_content:
    mycursor.execute(query, (title, authors, content))

# Commit the changes into the database
This updates the database, this time for real

In [13]:
text_db.commit()

# Print all rows of the table
Useful for checking the committed changes

In [11]:
# Execute a SELECT query
query = "SELECT * FROM chunks"
mycursor.execute(query)

# Fetch all rows
rows = mycursor.fetchall()

# Print the results
for row in rows:
    print(row)  # You can format this output as needed

(1, 1, 1, '\nON THE ELECTRODYNAMICS OF MOVING\nBODIES\n\nBy A. EINSTEIN\n\nJune 30, 1905\n\nIt is known that Maxwell’s electrodynamics—as usually understood at the\npresent time—when applied to moving bodies, leads to asymmetries which do\nnot appear to be inherent in the phe')
(2, 1, 2, 'nomena. Take, for example, the recipro-\neal electrodynamic action of a magnet and a conductor. The observable phe-\nnomenon here depends only on the relative motion of the conductor and the\nmagnet, whereas the customary view draws a sharp distinction betwe')
(3, 1, 3, 'en the two\ncases in which either the one or the other of these bodies is in motion. For if the\nmagnet is in motion and the conductor at rest, there arises in the neighbour-\nhood of the magnet an electric field with a certain definite energy, producing\na c')
(4, 1, 4, 'urrent at the places where parts of the conductor are situated. But if the\nmagnet is stationary and the conductor in motion, no electric field arises in the\nneighb

# Generate a dictionary from the paper
As of 09/06/2024, the chatbot model uses the dictionary data type to look up information,
so here I try to query the database and turn the paper we saved into a dictionary

In [17]:
# Execute a query to select all the rows from the 'papers' table
query = "SELECT title, author, chunk FROM papers"
mycursor.execute(query)

# Fetch all the rows
rows = mycursor.fetchall()

# Create a dictionary with the format you specified
paper = {
    "title": rows[0][0],
    "author": rows[0][1],
    "content": ''.join(row[2] for row in rows)
}

# Check our output
We verify if the paper has indeed been loaded into a dictionary

In [18]:
# Print the paper
print(paper)

{'title': 'A CASE STUDY OF UNDERSTANDING THE BONAPARTE BASIN USING UNSTRUCTURED DATA ANALYSIS WITH MACHINE LEARNING TECHNIQUES', 'author': 'A.N.N. Sazali, N.M. Hernandez, F. Baillard, K.G. Maver', 'content': "\nA CASE STUDY OF UNDERSTANDING THE\n\nBONAPARTE BASIN USING UNSTRUCTURED DATA\nANALYSIS WITH MACHINE LEARNING TECHNIQUES\n\nANN. Sazali!, N.M. Hernandez', F. Baillard', K.G. Maver!\n\n' Traya Energies\n\nSummary\n\nAs part of exploration and production the oil and gas industry produce substantial amounts of data\nwithin different disciplines of which 80% are unstructured like reports, presentations, spreadsheets etc.\nThe value of technical work is reduced due to the lack of time available for analysis and critical\nthinking and the under-utilization of the data. To assist geoscientist and engineers, Machine Learning\n(ML) and Artificial Intelligence (AI) technologies are applied to process the unstructured data from\n440 wells from the Bonaparte Basin in Australia making it poss